In [1]:
%matplotlib inline

In [2]:
from collections import defaultdict
from functools import partial
import math
import re

from matplotlib import pyplot as plt
import numexpr
import numpy as np
import pandas as pd

### Known issues
* Raiseskeleton life is high compared to game, but matches pet calculator
* Claygolem AR is 20 lower than game. Much lower than pet calculator, which uses character level.

In [3]:
CHARSKILL_EXCEPTIONS = ['mon death sentry', 'royalstrikemeteorfire']

N_PARAMS = 8
N_DESC_COLS = 6
N_DSC2_COLS = 4
N_DSC3_COLS = 7
N_SUBMISSILES_BY_ROOT = {
    'SubMissile': 3,
    'HitSubMissile': 4,
    'CltSubMissile': 3,
    'CltHitSubMissile': 4,
}

LVL_BREAKPOINTS = [(0, 1), (1, 8), (8, 16), (16, 22), (22, 28), (28, None)]

In [4]:
def get_string_mapped_columns():
    string_mapped_columns = ['str name', 'str long', 'str alt', 'str mana']
    for col_root, line_limit in {'desctext': N_DESC_COLS, 'dsc2text': N_DSC2_COLS, 'dsc3text': N_DSC3_COLS}.items():
        for a_b in 'ab':
            for i in range(1, line_limit + 1):
                string_mapped_columns.append(f'{col_root}{a_b}{i}')
    return string_mapped_columns

STRING_MAPPED_COLUMNS = get_string_mapped_columns()

In [5]:
def get_calc_columns():
    calc_columns = []
    for col_root, line_limit in {'desccalc': N_DESC_COLS, 'dsc2calc': N_DSC2_COLS, 'dsc3calc': N_DSC3_COLS}.items():
        for a_b in 'ab':
            for i in range(1, line_limit + 1):
                calc_columns.append(f'{col_root}{a_b}{i}')
    return calc_columns

CALC_COLUMNS = get_calc_columns()

# Read skills data
Extracted from patch_d2.mpq

* Corrected Firewall EDmgSymPerCalc, added )
* Adjusted Raise Skeleton descline, removed unneeded (), was breaking calc regexes

In [6]:
skills = pd.read_csv('data/skills.txt', delimiter='\t')
skilldesc = pd.read_csv('data/skilldesc.txt', delimiter='\t')

## Combine strings
Extracted from respective mpq files ENG localisation.

In [7]:
strings = pd.read_csv('data/string.csv', header=0, names=('key', 'value'))
expansionstring = pd.read_csv('data/expansionstring.csv', header=0, names=('key', 'value'))
patchstring = pd.read_csv('data/patchstring.csv', header=0, names=('key', 'value'))

In [8]:
all_strings = (
    pd.concat([
        strings.assign(source='strings').drop_duplicates(subset=['key'], keep='first'),
        expansionstring.assign(source='expansionstring').drop_duplicates(subset=['key'], keep='first'),
        patchstring.assign(source='patchstring').drop_duplicates(subset=['key'], keep='first'),
    ])
    .drop_duplicates(subset=['key'], keep='last')
)

In [9]:
all_strings.source.value_counts()

strings            4919
expansionstring    2700
patchstring        1061
Name: source, dtype: int64

### Prepare mappings from strings tables

In [10]:
def clean_string(s):
    try:
        cleaned = s.replace('\\n', '\n')
        if s[:2].isnumeric():
            return cleaned
        return '\n'.join(reversed(cleaned.split('\n')))  # reverse skill strings
    except AttributeError:
        return s

strings_map = (
    all_strings.set_index('key').value
    .apply(clean_string)
    .to_dict()
)

## Prepare additional tables

### Prepare mappings for EType

In [11]:
elem_types = pd.read_csv('data/ElemTypes.txt', delimiter='\t')
elemental_type_map = elem_types.dropna().set_index('Code')['Elemental Type'].to_dict()

### Prepare monster data

#### Monster stats

In [12]:
mon_stats = pd.read_csv('data/monstats.txt', delimiter='\t')

In [13]:
def get_monster_details(mon_stats):
    monster_details = {}
    for index, row in mon_stats.iterrows():
        row = row.copy().replace({np.nan: None})
        row_details = _get_monster_details_for_row(row)
        monster_details[row.Id] = {k: v for k, v in row_details.items() if v or v == 0}
    return monster_details

def _get_monster_details_for_row(row):
    return {
        'minHPNormal': safe_int(row['minHP']),
        'maxHPNormal': safe_int(row['maxHP']),
        'minHPNightmare': safe_int(row['MinHP(N)']),
        'maxHPNightmare': safe_int(row['MaxHP(N)']),
        'minHPHell': safe_int(row['MinHP(H)']),
        'maxHPHell': safe_int(row['MaxHP(H)']),
        'a1MinDNormal': safe_int(row['A1MinD']),
        'a1MaxDNormal': safe_int(row['A1MaxD']),
        'a1MinDNightmare': safe_int(row['A1MinD(N)']),
        'a1MaxDNightmare': safe_int(row['A1MaxD(N)']),
        'a1MinDHell': safe_int(row['A1MinD(H)']),
        'a1MaxDHell': safe_int(row['A1MaxD(H)']),
    }

def safe_int(x):
    try:
        return int(x)
    except (TypeError, ValueError):
        return x
    
monster_details = get_monster_details(mon_stats)

#### Monster lvl

In [14]:
mon_lvl = pd.read_csv('data/MonLvl.txt', delimiter='\t')

In [15]:
mon_lvl_column_renames = {
    'DM': 'DMNormal',
    'DM(N)': 'DMNightmare',
    'DM(H)': 'DMHell',
}
monster_level = mon_lvl.rename(columns=mon_lvl_column_renames).set_index('Level')[mon_lvl_column_renames.values()].to_dict()

### Prepare data from Missiles

In [16]:
missiles = pd.read_csv('data/Missiles.txt', delimiter='\t')

In [17]:
def get_missile_details(missiles):
    missile_details = {}
    for index, row in missiles.iterrows():
        row = row.copy().replace({np.nan: None})
        row_details = _get_missile_details_for_row(row)
        missile_details[row.Missile] = {k: v for k, v in row_details.items() if v or v == 0}
        
    for index, row in missiles.iterrows():  # requires all missiles populated first
        row = row.copy().replace({np.nan: None})
        for root, n_submissiles in N_SUBMISSILES_BY_ROOT.items():
            for submissile_num in range(1, n_submissiles + 1):
                submissile_key = row[f'{root}{submissile_num}']
                if submissile_key:
                    missile_details[row.Missile][f'{title_to_camelcase(root)}{submissile_num}'] = missile_details[submissile_key]
    return missile_details

def _get_missile_details_for_row(row):
    return {
        'range': safe_int(row.Range),
        'levRange': safe_int(row.LevRange),
        'hitShift': safe_int(row.HitShift),
        'minDamage': safe_int(row.MinDamage),
        'minLevDam1': safe_int(row.MinLevDam1),
        'minLevDam2': safe_int(row.MinLevDam2),
        'minLevDam3': safe_int(row.MinLevDam3),
        'minLevDam4': safe_int(row.MinLevDam4),
        'minLevDam5': safe_int(row.MinLevDam5),
        'maxDamage': safe_int(row.MaxDamage),
        'maxLevDam1': safe_int(row.MaxLevDam1),
        'maxLevDam2': safe_int(row.MaxLevDam2),
        'maxLevDam3': safe_int(row.MaxLevDam3),
        'maxLevDam4': safe_int(row.MaxLevDam4),
        'maxLevDam5': safe_int(row.MaxLevDam5),
        'dmgSymPerCalc': row.DmgSymPerCalc,
        'eType': row.EType,
        'eMin': safe_int(row.EMin),
        'eMinLev1': safe_int(row.MinELev1),
        'eMinLev2': safe_int(row.MinELev2),
        'eMinLev3': safe_int(row.MinELev3),
        'eMinLev4': safe_int(row.MinELev4),
        'eMinLev5': safe_int(row.MinELev5),
        'eMax': safe_int(row.Emax),
        'eMaxLev1': safe_int(row.MaxELev1),
        'eMaxLev2': safe_int(row.MaxELev2),
        'eMaxLev3': safe_int(row.MaxELev3),
        'eMaxLev4': safe_int(row.MaxELev4),
        'eMaxLev5': safe_int(row.MaxELev5),
        'eDmgSymPerCalc': row.EDmgSymPerCalc,
        'eLen': safe_int(row.ELen),
        'eLevLen1': safe_int(row.ELevLen1),
        'eLevLen2': safe_int(row.ELevLen2),
        'eLevLen3': safe_int(row.ELevLen3),
    }

def title_to_camelcase(s):
    if not s:
        return ''
    if ' ' in s:
        s = s.title()
    return (s[:1].lower() + s[1:]).replace(' ', '')
    
missile_details = get_missile_details(missiles)

# Merge descriptions for character skills

In [18]:
charskills = (
    skills
    .loc[skills.charclass.notnull() | (skills.skill.isin(CHARSKILL_EXCEPTIONS))]
    .merge(skilldesc, how='left', on='skilldesc', validate='one_to_one')
)
# Replace string keys with values from .tbl files
charskills.loc[:, STRING_MAPPED_COLUMNS] = charskills[STRING_MAPPED_COLUMNS].replace(strings_map)
# Replace elemental types with mapped values
charskills.loc[:, 'EType'] = charskills.EType.replace(elemental_type_map)

# Create skill details lookup

In [19]:
def get_skill_details(charskills):
    skill_details = {}
    for index, row in charskills.iterrows():
        row = row.copy().replace({np.nan: None})
        skill_key = title_to_camelcase(row.skill)

        row_details = _get_skill_details_for_row(row)
        skill_details[skill_key] = {k: v for k, v in row_details.items() if v or v == 0}
        
    for index, row in charskills.iterrows():  # requires all skills populated first
        row = row.copy().replace({np.nan: None})
        skill_key = title_to_camelcase(row.skill)

        related_skills = _get_related_entities_for_calcs(row, r"(?:skill|sklvl)\('((?:\w|\s)+)'(?:\.(?!lvl)\w+)+\)")
        skill_details[skill_key]['relatedSkills'] = {skill: _without_related(skill_details[skill]) for skill in related_skills}
        
        related_missiles = _get_related_entities_for_calcs(row, r"miss\('((?:\w|\s)+)'\.(?!lvl)\w+\)")
        skill_details[skill_key]['relatedMissiles'] = {missile: _without_related(missile_details[missile]) for missile in related_missiles}

    return skill_details

def _get_related_entities_for_calcs(row, pattern):
    related = set()
    for calc_column in CALC_COLUMNS:
        calc_expression = row[calc_column]
        if not calc_expression:
            continue

        matches = re.findall(pattern=pattern, string=str(calc_expression))
        related = related.union({title_to_camelcase(match) for match in matches})
    return related
    
    
def _without_related(skill):
    skill = skill.copy()
    skill.pop('relatedSkills', None)
    skill.pop('relatedMissiles', None)
    return skill
    

def _get_skill_details_for_row(row):
    return {
        'strName': row['str name'],
        'strLong': row['str long'],
        'skillPage': safe_int(row.SkillPage),
        'skillRow': safe_int(row.SkillRow),
        'skillColumn': safe_int(row.SkillColumn),
        'strMana': row['str mana'],
        'mana': safe_int(row['mana']),
        'lvlMana': safe_int(row['lvlmana']),
        'minMana': safe_int(row['minmana']),
        'manaShift': safe_int(row['manashift']),
        'toHit': safe_int(row.ToHit),
        'levToHit': safe_int(row.LevToHit),
        'toHitCalc': row.ToHitCalc,
        'hitShift': safe_int(row.HitShift),
        'srcDam': safe_int(row.SrcDam),
        'minDam': safe_int(row.MinDam),
        'minLevDam1': safe_int(row.MinLevDam1),
        'minLevDam2': safe_int(row.MinLevDam2),
        'minLevDam3': safe_int(row.MinLevDam3),
        'minLevDam4': safe_int(row.MinLevDam4),
        'minLevDam5': safe_int(row.MinLevDam5),
        'maxDam': safe_int(row.MaxDam),
        'maxLevDam1': safe_int(row.MaxLevDam1),
        'maxLevDam2': safe_int(row.MaxLevDam2),
        'maxLevDam3': safe_int(row.MaxLevDam3),
        'maxLevDam4': safe_int(row.MaxLevDam4),
        'maxLevDam5': safe_int(row.MaxLevDam5),
        'dmgSymPerCalc': row.DmgSymPerCalc,
        'eType': row.EType,
        'eMin': safe_int(row.EMin),
        'eMinLev1': safe_int(row.EMinLev1),
        'eMinLev2': safe_int(row.EMinLev2),
        'eMinLev3': safe_int(row.EMinLev3),
        'eMinLev4': safe_int(row.EMinLev4),
        'eMinLev5': safe_int(row.EMinLev5),
        'eMax': safe_int(row.EMax),
        'eMaxLev1': safe_int(row.EMaxLev1),
        'eMaxLev2': safe_int(row.EMaxLev2),
        'eMaxLev3': safe_int(row.EMaxLev3),
        'eMaxLev4': safe_int(row.EMaxLev4),
        'eMaxLev5': safe_int(row.EMaxLev5),
        'eDmgSymPerCalc': row.EDmgSymPerCalc,
        'eLen': safe_int(row.ELen),
        'eLevLen1': safe_int(row.ELevLen1),
        'eLevLen2': safe_int(row.ELevLen2),
        'eLevLen3': safe_int(row.ELevLen3),
        'eLenSymPerCalc': row.ELenSymPerCalc,
        'params': {f'par{i}': safe_int(row[f'Param{i}']) for i in range(1, N_PARAMS + 1) if row.get(f'Param{i}')},
        'descLines': _get_desclines_for_row(row, 'desc', N_DESC_COLS),
        'dsc2Lines': _get_desclines_for_row(row, 'dsc2', N_DSC2_COLS),
        'dsc3Lines': _get_desclines_for_row(row, 'dsc3', N_DSC3_COLS),
        'missile1': missile_details.get(row.descmissile1),
        'missile2': missile_details.get(row.descmissile2),
        'missile3': missile_details.get(row.descmissile3),
        'summon': monster_details.get(row.summon.lower() if row.summon else None),
    }
    
def _get_desclines_for_row(row, column_root, max_entries):
    row = row.copy().replace({np.nan: None})
    
    entries = []
    for i in range(1, max_entries + 1):
        if not row[f'{column_root}line{i}']:
            continue

        entry = {
            f'{column_root}Line': int(row[f'{column_root}line{i}']),
            f'{column_root}TextA': row[f'{column_root}texta{i}'],
            f'{column_root}TextB': row[f'{column_root}textb{i}'],
            f'{column_root}CalcA': safe_int(row[f'{column_root}calca{i}']),
            f'{column_root}CalcB': safe_int(row[f'{column_root}calcb{i}']),
        }
        entries.append({k: v for k, v in entry.items() if v or v == 0})
        
    if column_root in ('desc', 'dsc2'):
        entries.reverse()  # D2 renders desclines bottom-up
    return entries

In [20]:
skill_details = get_skill_details(charskills)

# Generate descriptions from lookup

## Define calculators

In [21]:
def calculate_damage(skill, skill_levels, lvl, initial_damage_key, damage_per_level_key_root, synergy_key, missile_num=None):
    if missile_num:
        skill = skill[f'missile{missile_num}']
    
    synergy_bonus = calculate_skill_value(skill.get(synergy_key), skill, skill_levels, lvl) or 0
    synergy_multiplier = (100 + synergy_bonus) / 100
    
    damage_per_level_values = (
        [skill[initial_damage_key]] + [skill.get(f'{damage_per_level_key_root}{i}', 0) for i in range(1, 6)]
    )
    damage = 0
    for (lower, upper), damage_per_level in zip(LVL_BREAKPOINTS, damage_per_level_values):
        if lvl <= lower:
            break
        lvl_for_band = min(upper, lvl) if upper is not None else lvl
        damage += (lvl_for_band - lower) * damage_per_level
    hitshift = skill.get('hitShift', 0)
    return damage * synergy_multiplier * 2 ** (hitshift - 8)

calculate_physical_damage_min = partial(
    calculate_damage,
    initial_damage_key='minDam',
    damage_per_level_key_root='minLevDam',
    synergy_key='dmgSymPerCalc',
)
calculate_physical_damage_max = partial(
    calculate_damage,
    initial_damage_key='maxDam',
    damage_per_level_key_root='maxLevDam',
    synergy_key='dmgSymPerCalc',
)
calculate_elemental_damage_min = partial(
    calculate_damage,
    initial_damage_key='eMin',
    damage_per_level_key_root='eMinLev',
    synergy_key='eDmgSymPerCalc',
)
calculate_elemental_damage_max = partial(
    calculate_damage,
    initial_damage_key='eMax',
    damage_per_level_key_root='eMaxLev',
    synergy_key='eDmgSymPerCalc',
)

In [22]:
def calculate_mana_cost_per_second(skill, skill_levels, lvl):
    return calculate_mana_cost(skill, skill_levels, lvl) * FRAMES_PER_SECOND / 2

def calculate_mana_cost(skill, skill_levels, lvl):
    mana, lvl_mana, mana_shift = skill['mana'], skill.get('lvlMana', 0), skill['manaShift']
    cost = (mana + lvl_mana * (lvl - 1)) * 2 ** (mana_shift - 8)
    return max(cost, skill.get('minMana', 0))

def floor(x, precision=None):
    if precision is None:
        return math.floor(x)
    floored = math.floor(precision * x) / precision
    return int_if_no_decimals(floored)

def int_if_no_decimals(x):
    return int(x) if not x % 1 else x

def calculate_to_hit(skill, skill_levels, lvl):
    to_hit_expression = skill.get('toHitCalc')
    if to_hit_expression:
        return calculate_skill_value(to_hit_expression, skill, skill_levels, lvl)

    return skill.get('toHit', 0) + skill.get('levToHit', 0) * (lvl - 1)

def calculate_length(skill, skill_levels, lvl):
    synergy_length_bonus = calculate_skill_value(skill.get('eLenSymPerCalc'), skill, skill_levels, lvl) or 0
    synergy_length_multiplier = (100 + synergy_length_bonus) / 100

    length_per_level_values = (
        [skill['eLen']] + [skill.get(f'eLevLen{i}', 0) for i in range(1, 4)]
    )
    length = 0
    for (lower, upper), length_per_level in zip(LVL_BREAKPOINTS[:3], length_per_level_values):
        if lvl <= lower:
            break
        lvl_for_band = min(upper, lvl) if upper is not None else lvl
        length += (lvl_for_band - lower) * length_per_level

    return length * synergy_length_multiplier

def calculate_mXeo(skill, skill_levels, lvl, missile_num):
    return 256 * calculate_elemental_damage_min(skill, skill_levels, lvl, missile_num=missile_num)

def calculate_mXey(skill, skill_levels, lvl, missile_num):
    return 256 * calculate_elemental_damage_max(skill, skill_levels, lvl, missile_num=missile_num)

def create_linear_calculator(param_key_a, param_key_b):
    def calculator(skill, skill_levels, lvl):
        a = skill['params'].get(param_key_a, 0)
        b = skill['params'].get(param_key_b, 0)
        return a + b * (lvl - 1)
    return calculator

def create_diminishing_calculator(param_key_a, param_key_b):
    def calculator(skill, skill_levels, lvl):
        a = skill['params'].get(param_key_a, 0)
        b = skill['params'].get(param_key_b, 0)
        return floor(a + ((110 * lvl) * (b - a)) / (100 * (lvl + 6)))
    return calculator

def _get_param(skill, skill_levels, lvl, param_number):
    return skill['params'][f'par{param_number}']

In [23]:
def calculate_skill_value(calc_expression, skill, skill_levels, lvl):
    if calc_expression is None:
        return None

    calc_expression = _fill_other_skill_levels(str(calc_expression), skill_levels)
    calc_expression = _evaluate_sklvl_calcs(calc_expression, skill, skill_levels, lvl)
    calc_expression = _evaluate_other_entity_calcs(calc_expression, skill, skill_levels, lvl)
    calc_expression = _evaluate_calcs(calc_expression, skill, skill_levels, lvl)
    calc_expression = _evaluate_mins(calc_expression)
    calc_expression = _fill_if_thens(calc_expression)

    calc = numexpr.evaluate(calc_expression).item()
    return calc

def _fill_other_skill_levels(calc_expression, skill_levels):
    return re.sub(
        pattern=r"skill\('((?:\w|\s)+)'.(?:lvl|blvl)\)",
        repl=partial(_matched_skill_level, skill_levels=skill_levels),
        string=calc_expression,
    )

def _evaluate_sklvl_calcs(calc_expression, skill, skill_levels, lvl):
    return re.sub(
        pattern=r"sklvl\('((?:\w|\s)+)'\.(\w+)\.(?!lvl)(\w+)\)",
        repl=partial(_matched_sklvl_calc, skill=skill, skill_levels=skill_levels, lvl=lvl),
        string=calc_expression,
    )

def _evaluate_other_entity_calcs(calc_expression, skill, skill_levels, lvl):
    return re.sub(
        pattern=r"(skill|miss)\('((?:\w|\s)+)'\.(?!lvl)(\w+)\)",
        repl=partial(_matched_other_entity_calc, skill=skill, skill_levels=skill_levels, lvl=lvl),
        string=calc_expression,
    )

def  _evaluate_calcs(calc_expression, skill, skill_levels, lvl):
    return re.sub(
        pattern='|'.join(CALC_LOOKUP.keys()),
        repl=partial(_matched_calc, skill=skill, skill_levels=skill_levels, lvl=lvl),
        string=calc_expression,
    )

def _fill_if_thens(calc_expression):
    return re.sub(
        pattern=r'(.*)\s\?\s?(.*)\s?:\s?(.*)',
        repl=lambda match: match[2] if numexpr.evaluate(match[1]).item() else match[3],
        string=calc_expression,
    )

def _evaluate_mins(calc_expression):
    RE_MATH_SYMBOLS = '(?:\d|\+|-|\*|\/|\.|\s|\(|\))'
    return re.sub(
        pattern=fr"min\(({RE_MATH_SYMBOLS}+),\s*({RE_MATH_SYMBOLS}+)\)",
        repl=_matched_min_function,
        string=calc_expression,
    )

def _matched_skill_level(match, skill_levels):
    skill_name = title_to_camelcase(match[1])
    return str(skill_levels.get(skill_name, 0))

def _matched_sklvl_calc(match, skill, skill_levels, lvl):
    other_skill_name = title_to_camelcase(match[1])
    other_skill = skill[f'relatedSkills'][other_skill_name]
    
    lvl_calculator = CALC_LOOKUP[match[2]]
    calculator = CALC_LOOKUP[match[3]]

    effective_lvl = lvl_calculator(skill, skill_levels, lvl)
    return str(calculator(other_skill, skill_levels, effective_lvl))

def _matched_other_entity_calc(match, skill, skill_levels, lvl):
    entity_kind = {'miss': 'Missile', 'skill': 'Skill'}[match[1]]
    entity_name = title_to_camelcase(match[2])
    entity = skill[f'related{entity_kind}s'][entity_name]
    
    calculator = CALC_LOOKUP[match[3]]
    lvl = skill_levels.get(entity_name, 0) if entity_kind == 'Skill' else lvl
    return str(calculator(entity, skill_levels, lvl))
        
def _matched_calc(match, skill, skill_levels, lvl):
    calculator = CALC_LOOKUP[match[0]]
    return str(calculator(skill, skill_levels, lvl))

def _matched_min_function(match):
    return min(match[1], match[2])

In [24]:
CALC_LOOKUP = {
    'toht': calculate_to_hit,
    'edmn': calculate_elemental_damage_min,
    'edmx': calculate_elemental_damage_max,
    'edln': calculate_length,
    'edns': lambda skill, skill_levels, lvl: 256 * calculate_elemental_damage_min(skill, skill_levels, lvl),
    'edxs': lambda skill, skill_levels, lvl: 256 * calculate_elemental_damage_max(skill, skill_levels, lvl),
    'ln12': create_linear_calculator('par1', 'par2'),
    'ln34': create_linear_calculator('par3', 'par4'),
    'ln56': create_linear_calculator('par5', 'par6'),
    'ln78': create_linear_calculator('par7', 'par8'),
    'dm12': create_diminishing_calculator('par1', 'par2'),
    'dm34': create_diminishing_calculator('par3', 'par4'),
    'dm56': create_diminishing_calculator('par5', 'par6'),
    'dm78': create_diminishing_calculator('par7', 'par8'),
    'mps': calculate_mana_cost_per_second,
    'm1eo': partial(calculate_mXeo, missile_num=1),
    'm1ey': partial(calculate_mXey, missile_num=1),
    'm2eo': partial(calculate_mXeo, missile_num=2),
    'm2ey': partial(calculate_mXey, missile_num=2),
    'm3eo': partial(calculate_mXeo, missile_num=3),
    'm3ey': partial(calculate_mXey, missile_num=3),
    'm1en': partial(calculate_elemental_damage_min, missile_num=1),
    'm1ex': partial(calculate_elemental_damage_max, missile_num=1),
    'm2en': partial(calculate_elemental_damage_min, missile_num=2),
    'm2ex': partial(calculate_elemental_damage_max, missile_num=2),
    'm3en': partial(calculate_elemental_damage_min, missile_num=3),
    'm3ex': partial(calculate_elemental_damage_max, missile_num=3),
    'lvl': lambda skill, skill_levels, lvl: lvl,
    **{f'par{i}': partial(_get_param, param_number=i) for i in range(1, 9)},
}

## Define formatters

In [25]:
FRAMES_PER_SECOND = 25
YARDS_PER_GAME_UNIT = 2 / 3

def format_mana_cost(skill, skill_levels, lvl, ta, tb, ca, cb):
    cost = floor(calculate_mana_cost(skill, skill_levels, lvl), precision=ca or 10)
    return f"{skill['strMana']}{cost}" if cost else ''

def format_attack_rating(skill, skill_levels, lvl, ta, tb, ca, cb):
    attack_rating = calculate_to_hit(skill, skill_levels, lvl)
    if not attack_rating:
        return ''
    return f"Attack: +{attack_rating} percent"

def format_physical_damage(skill, skill_levels, lvl, ta, tb, ca, cb):
    if skill.get('minDam') is None:
        return ''

    min_damage = floor(calculate_physical_damage_min(skill, skill_levels, lvl))
    max_damage = floor(calculate_physical_damage_max(skill, skill_levels, lvl))

    if min_damage == max_damage:
        return f'Damage: +{min_damage}'
    
    return f'Damage: {min_damage}-{max_damage}'

def format_elemental_damage(skill, skill_levels, lvl, ta, tb, ca, cb):
    if skill.get('eMin') is None:
        return ''
    
    min_damage = floor(calculate_elemental_damage_min(skill, skill_levels, lvl))
    max_damage = floor(calculate_elemental_damage_max(skill, skill_levels, lvl))

    if min_damage == max_damage:
        return f"{skill['eType']} Damage: +{min_damage}"
    
    return f"{skill['eType']} Damage: {min_damage}-{max_damage}"

def format_elemental_damage_with_text(skill, skill_levels, lvl, ta, tb, ca, cb):
    if skill.get('eMin') is None:
        return ''
    
    min_damage = floor(calculate_elemental_damage_min(skill, skill_levels, lvl))
    max_damage = floor(calculate_elemental_damage_max(skill, skill_levels, lvl))
    
    return f"{tb or ''}{ta or ''} {min_damage}-{max_damage}"

def format_elemental_length(skill, skill_levels, lvl, ta, tb, ca, cb):
    if not skill.get('eLen'):
        return ''
    
    length = calculate_length(skill, skill_levels, lvl)
    formatted_length = int_if_no_decimals(length / FRAMES_PER_SECOND)
    return f"{skill['eType']} Length: {formatted_length} seconds"

def format_poison_damage(skill, skill_levels, lvl, ta, tb, ca, cb):
    if skill.get('eMin') is None:
        return ''
    
    adjustment = 1 if skill.get('hitShift') else 2 ** -8  # skills without hitshift 
    
    length = calculate_length(skill, skill_levels, lvl)
    min_damage = floor(length * calculate_elemental_damage_min(skill, skill_levels, lvl))
    max_damage = floor(length * calculate_elemental_damage_max(skill, skill_levels, lvl))
    formatted_length = int_if_no_decimals(length / FRAMES_PER_SECOND)
    return f'Poison Damage: {min_damage}-{max_damage}\nover {formatted_length} seconds'

def format_elemental_damage_over_time(skill, skill_levels, lvl, ta, tb, ca, cb):
    min_damage = floor(FRAMES_PER_SECOND * calculate_elemental_damage_min(skill, skill_levels, lvl))
    max_damage = floor(FRAMES_PER_SECOND * calculate_elemental_damage_max(skill, skill_levels, lvl))
    return f"Average {skill['eType']} Damage: {min_damage}-{max_damage} per second"

def format_elemental_damage_over_time_with_text(skill, skill_levels, lvl, ta, tb, ca, cb):
    calca = floor(calculate_skill_value(ca, skill, skill_levels, lvl))
    calcb = floor(calculate_skill_value(cb, skill, skill_levels, lvl))
    return f"{tb or ''}{ta or ''}{calca}-{calcb} per second"

def format_fire_missile_damage_over_time(skill, skill_levels, lvl, ta, tb, ca, cb):
    adjustment = 2.34 * 2 ** 5  # tuned, just higher than 7/3
    min_damage = floor(adjustment * calculate_elemental_damage_min(skill, skill_levels, lvl, missile_num=1))
    max_damage = floor(adjustment * calculate_elemental_damage_max(skill, skill_levels, lvl, missile_num=1))
    return f'Average Fire Damage: {min_damage}-{max_damage} per second'

def format_fire_damage_over_time(skill, skill_levels, lvl, ta, tb, ca, cb):
    adjustment = 2.34 * 2 ** 5  # tuned, just higher than 7/3
    min_damage = floor(adjustment * calculate_elemental_damage_min(skill, skill_levels, lvl))
    max_damage = floor(adjustment * calculate_elemental_damage_max(skill, skill_levels, lvl))
    return f'Average Fire Damage: {min_damage}-{max_damage} per second'

def format_missile_range_in_yards(skill, skill_levels, lvl, ta, tb, ca, cb):
    missile_range = skill['missile1']['range'] + skill['missile1'].get('levRange', 0) * (lvl - 1)
    formatted_range = floor(missile_range * YARDS_PER_GAME_UNIT, 10)
    return f'{formatted_range} yards'

def format_missile_duration(skill, skill_levels, lvl, ta, tb, ca, cb):
    duration = skill['missile1']['range'] + skill['missile1'].get('levRange', 0) * (lvl - 1)
    formatted_duration = int_if_no_decimals(duration / FRAMES_PER_SECOND)
    return f'{ta}{formatted_duration} seconds'

def format_cltsubmissile1_duration(skill, skill_levels, lvl, ta, tb, ca, cb):
    missile = skill['missile1']['cltSubMissile1']
    duration = missile['range'] + missile.get('levRange', 0) * (lvl - 1)
    formatted_duration = int_if_no_decimals(duration / FRAMES_PER_SECOND)
    return f'{ta}{formatted_duration} seconds'

def format_minion_life(skill, skill_levels, lvl, ta, tb, ca, cb):
    base_life = (skill['summon']['minHPNormal'] + skill['summon']['maxHPNormal']) / 2
    calca = calculate_skill_value(ca, skill, skill_levels, lvl) or 0
    calcb = calculate_skill_value(cb, skill, skill_levels, lvl) or 0
    
    life = floor((1 + calca / 100) * (base_life + calcb))
    return f'Life: {life}'

def format_skeleton_damage(skill, skill_levels, lvl, ta, tb, ca, cb):
    mastery_damage = skill_levels['skeletonMastery'] * skill['relatedSkills']['skeletonMastery']['params']['par2']
    monster_min_damage = skill['summon']['a1MinDNormal']
    monster_max_damage = skill['summon']['a1MaxDNormal']
    
    skill_damage = calculate_elemental_damage_min(skill, skill_levels, lvl)
    bonus = (lvl - 3) * skill['params']['par3'] if lvl > 3 else 0
    multiplier = 1 + bonus / 100

    min_damage = floor(multiplier * (monster_min_damage + skill_damage + mastery_damage))
    max_damage = floor(multiplier * (monster_max_damage + skill_damage + mastery_damage))

    return f'Damage: {min_damage}-{max_damage}'

def format_golem_damage(skill, skill_levels, lvl, ta, tb, ca, cb):
    damage_bonus = 0 if skill['strName'] == 'Iron Golem' else 35 * (lvl - 1)
    damage_multiplier = 1 + damage_bonus / 100
    
    monster_min_damage = skill['summon']['a1MinDNormal']
    monster_max_damage = skill['summon']['a1MaxDNormal']
    
    min_damage = floor(damage_multiplier * monster_min_damage)
    max_damage = floor(damage_multiplier * monster_max_damage)
    return f'Damage: {min_damage}-{max_damage}'

def format_fire_golem_damage(skill, skill_levels, lvl, ta, tb, ca, cb):
    monster_min_damage = skill['summon']['a1MinDNormal']
    monster_max_damage = skill['summon']['a1MaxDNormal']
    
    holy_fire_min_damage = calculate_skill_value(ca, skill, skill_levels, lvl)
    holy_fire_max_damage = calculate_skill_value(cb, skill, skill_levels, lvl)
    
    min_damage = floor(monster_min_damage + holy_fire_min_damage)
    max_damage = floor(monster_max_damage + holy_fire_max_damage)
    return f'Fire Damage: {min_damage}-{max_damage}'

def format_half_radius(skill, skill_levels, lvl, ta, tb, ca, cb):
    calca = calculate_skill_value(ca, skill, skill_levels, lvl)
    radius = floor(floor(calca) * YARDS_PER_GAME_UNIT / 2, precision=10)
    return f'{ta}{radius} yards'

def format_256_precision(skill, skill_levels, lvl, ta, tb, ca, cb):
    calca = calculate_skill_value(ca, skill, skill_levels, lvl)
    calcb = calculate_skill_value(cb, skill, skill_levels, lvl)
    min_damage = floor(calca / 256, precision=10)
    max_damage = floor(calcb / 256, precision=10)
    return f"{ta or ''}{min_damage}-{max_damage}{tb or ''}"

def replace_ta_with_calc_on_ca(skill, skill_levels, lvl, ta, tb, ca, cb, pattern='%d%'):
    return ta.replace(pattern, str(calculate_skill_value(ca, skill, skill_levels, lvl)))

def format_text_on_calc_condition(skill, skill_levels, lvl, ta, tb, ca, cb):
    calca = floor(calculate_skill_value(ca, skill, skill_levels, lvl))
    return f'{calca}{ta if calca == 1 else tb}'

def create_calc_formatter(template, frames=False, game_units=False, precision=None):
    def formatter(skill, skill_levels, lvl, ta, tb, ca, cb):
        calca = calculate_skill_value(ca, skill, skill_levels, lvl)
        calcb = calculate_skill_value(cb, skill, skill_levels, lvl)
        
        calca = _convert_calc(calca, frames, game_units, precision)
        calcb = _convert_calc(calcb, frames, game_units, precision)

        formatted = template.format(ta=ta or '', tb=tb or '', calca=calca, calcb=calcb)
        return formatted if calca or calcb else ''
    return formatter

def _convert_calc(calc, frames, game_units, precision):
    if calc is None:
        return calc
    if frames:
        calc /= FRAMES_PER_SECOND
    if game_units:
        calc = floor(calc) * YARDS_PER_GAME_UNIT
    if precision is not None:
        calc = floor(calc, precision)
    return calc

In [26]:
FORMATTERS_BY_DESCLINE = {
    1: format_mana_cost,
    2: create_calc_formatter('{ta} {calca:+d}{tb}'),
    3: create_calc_formatter('{ta}{calca}{tb}', precision=1),
    4: create_calc_formatter('{ta}+{calca}'),
    5: create_calc_formatter('{ta} {calca}', precision=1),
    6: create_calc_formatter('+{calca} {ta}'),
    7: create_calc_formatter('{calca} {ta}'),
    8: format_attack_rating,
    9: format_physical_damage,
    10: format_elemental_damage,
    11: format_elemental_length,
    12: create_calc_formatter('{ta}{calca} seconds', frames=True, precision=10),
    13: format_minion_life,
    14: format_poison_damage,
    15: lambda skill, skill_levels, lvl, ta, tb, ca, cb: f"{ta or ''}:{tb or ''}",  # not used
    16: create_calc_formatter('Duration: {calca}-{calcb} seconds', frames=True),
    17: format_elemental_damage_over_time_with_text,
    18: lambda skill, skill_levels, lvl, ta, tb, ca, cb: ta,
    19: create_calc_formatter('{tb}{ta} {calca} yards', game_units=True, precision=10),
    20: create_calc_formatter('{ta}+{calca} percent {tb}'),  # not used
    21: create_calc_formatter('{ta}{calca} percent {tb}'),  # not used
    22: format_fire_missile_damage_over_time,
    23: format_missile_duration,
    24: format_elemental_damage_with_text,
    25: lambda skill, skill_levels, lvl, ta, tb, ca, cb: f"{ta or ''}{tb or ''}",
    26: format_elemental_damage_over_time,
    27: format_fire_damage_over_time,
    28: lambda skill, skill_levels, lvl, ta, tb, ca, cb: 'Radius: 1 yard',
    29: format_missile_range_in_yards,
    30: format_cltsubmissile1_duration,
    31: create_calc_formatter('{ta} {calca} seconds', frames=True, precision=10),
    32: create_calc_formatter('{ta}{tb}+{calca} percent'),
    33: lambda skill, skill_levels, lvl, ta, tb, ca, cb: f'{ta}{tb}',
    34: format_skeleton_damage,
    35: create_calc_formatter('{ta}: {calca}-{calcb}', precision=1),
    36: format_text_on_calc_condition,
    37: format_half_radius,
    38: create_calc_formatter('{ta}{calca}-{calcb}{tb}', precision=1),
    39: format_golem_damage,
    40: lambda skill, skill_levels, lvl, ta, tb, ca, cb: ta.replace('%s', tb),  # ca refers to color, ignored here
    41: format_fire_golem_damage,
    42: create_calc_formatter('{ta}: +{calca}.{calcb} {tb}', precision=1),
    43: format_256_precision,
    44: format_256_precision,  # not used
    # 45: not used
    46: lambda skill, skill_levels, lvl, ta, tb, ca, cb: ta,  # not used
    
    59: create_calc_formatter('{tb}{ta}{calca}-{calcb}', precision=1),
    
    62: create_calc_formatter('{ta}{tb}{calca}-{calcb}', precision=1),
    63: create_calc_formatter('{ta}: +{calca}% {tb}'),

    66: replace_ta_with_calc_on_ca,
    67: create_calc_formatter('{ta}: +{calca} {tb}'),

    70: create_calc_formatter('{ta}{tb}+{calca}'),
    71: lambda skill, skill_levels, lvl, ta, tb, ca, cb: f'{ta}: {tb}',
    72: create_calc_formatter('+{calca}/{calcb} {ta}'),
    73: create_calc_formatter('{calca}/{calcb} {ta}'),
}

def format_descline(descline_number, skill, skill_levels, lvl, ta, tb, ca, cb):
    formatter = FORMATTERS_BY_DESCLINE[descline_number]
    return formatter(skill, skill_levels, lvl, ta, tb, ca, cb)

## Format skills

In [31]:
def format_skill(skill, skill_levels, lvl):
    lines = [
        skill['strName'],
        skill['strLong'],
        '',
        format_desclines('dsc2', skill, skill_levels, lvl),
        f'Current Level: {lvl}',
        format_desclines('desc', skill, skill_levels, lvl),
        f'Next Level: {lvl + 1}',
        format_desclines('desc', skill, skill_levels, lvl + 1),
        format_desclines('dsc3', skill, skill_levels, lvl),
    ]
    return '\n'.join([line for line in lines if line is not None])

def format_desclines(root, skill, skill_levels, lvl):
    lines = []
    for entry in skill.get(f'{root}Lines', []):
        line = format_descline(
            descline_number=entry[f'{root}Line'],
            skill=skill,
            skill_levels=skill_levels,
            lvl=lvl,
            ta=entry.get(f'{root}TextA'),
            tb=entry.get(f'{root}TextB'),
            ca=entry.get(f'{root}CalcA'),
            cb=entry.get(f'{root}CalcB'),
        )
        if line:
            lines.append(line)
    return '\n'.join(lines) + '\n' if lines else None

print(format_skill(skill_details['holyBolt'], {'clayGolem': 1}, 10))

KeyError: 47

In [30]:
skill_details['holyBolt']

{'strName': 'Holy Bolt',
 'strLong': 'a bolt of divine energy\nthat damages undead enemies\nor heals allies',
 'skillPage': 1,
 'skillRow': 2,
 'skillColumn': 2,
 'strMana': 'Mana Cost: ',
 'mana': 32,
 'lvlMana': 1,
 'minMana': 1,
 'manaShift': 4,
 'hitShift': 8,
 'eType': 'Magic',
 'eMin': 8,
 'eMinLev1': 8,
 'eMinLev2': 10,
 'eMinLev3': 13,
 'eMinLev4': 16,
 'eMinLev5': 20,
 'eMax': 16,
 'eMaxLev1': 8,
 'eMaxLev2': 11,
 'eMaxLev3': 15,
 'eMaxLev4': 18,
 'eMaxLev5': 23,
 'eDmgSymPerCalc': "(skill('Blessed Hammer'.blvl)+skill('Fist of the Heavens'.blvl))*par8",
 'params': {'par1': 1,
  'par2': 2,
  'par3': 6,
  'par4': 4,
  'par7': 15,
  'par8': 50},
 'descLines': [{'descLine': 10},
  {'descLine': 47,
   'descTextA': 'Heals: ',
   'descCalcA': "ln12 * (100 + skill('Prayer'.blvl) * par7) / 100",
   'descCalcB': "ln34 * (100 + skill('Prayer'.blvl) * par7) / 100"},
  {'descLine': 1}],
 'dsc3Lines': [{'dsc3Line': 40,
   'dsc3TextA': '%s Receives Bonuses From:',
   'dsc3TextB': 'Holy Bolt'

## Look up skills by descline for development

In [29]:
def select_skills_with_descline(charskills, pattern):
    descline_columns = get_descline_columns()
    return charskills.loc[(charskills[descline_columns] == pattern).any(axis=1), ['skill'] + descline_columns]

def get_descline_columns():
    descline_columns = []
    for col_root, line_limit in {'desc': N_DESC_COLS, 'dsc2': N_DSC2_COLS, 'dsc3': N_DSC3_COLS}.items():
        for i in range(1, line_limit + 1):
            descline_columns.append(f'{col_root}line{i}')
    return descline_columns

select_skills_with_descline(charskills, 47)

,skill,descline1,descline2,descline3,descline4,descline5,descline6,dsc2line1,dsc2line2,dsc2line3,dsc2line4,dsc3line1,dsc3line2,dsc3line3,dsc3line4,dsc3line5,dsc3line6,dsc3line7
95,Holy Bolt,1.0,47.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0,63.0,63.0,63.0,NaN,NaN,NaN
